In [ ]:
!pip install rouge sumy

In [ ]:
import sys
import nltk
import pandas as pd
from rouge import Rouge
from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.luhn import LuhnSummarizer

In [ ]:
score_path = "/content/drive/MyDrive/LegSuM/scores/"

In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/catest_processed.csv"
data = pd.read_csv(path)

In [ ]:
data.sample(5)

,bill_id,clean_text,summary,sum_len,text_len,cleantext_len
810,AB 1382,The people of the State of California do enact...,Existing law makes it a crime for a person who...,659,7490,7407
366,SB 1339,The people of the State of California do enact...,Existing law provides for the California Work ...,4695,16168,15588
367,SB 1341,The people of the State of California do enact...,"Existing law, as amended by Proposition 9, the...",1190,6548,6473
498,AB 381,The people of the State of California do enact...,"Existing law, in cases in which a minor is adj...",2293,8936,8763
540,AB 541,The people of the State of California do enact...,Existing law provides for the establishment of...,1970,12519,11961


In [ ]:
nltk.download("punkt")
rouge = Rouge()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)
print(sys.getrecursionlimit())

1000
5000


In [ ]:
def summarize(text, sumarizer, SENTENCES_COUNT):
    sentences_ = []
    doc = text
    doc_ = PlaintextParser(doc, Tokenizer("en")).document
    for sentence in sumarizer(doc_, SENTENCES_COUNT):
        sentences_.append(str(sentence))

    summm_ = " ".join(sentences_)
    return summm_

In [ ]:
data["LuhnSummary"] = data["clean_text"].map(
                                  lambda x: summarize(x, LuhnSummarizer(), 5)
                                    )

In [ ]:
data[['summary','LuhnSummary']].sample(5)

,summary,LuhnSummary
807,The Psychology Licensing Law provides for the ...,A person other than a licensed psychologist ma...
591,"Under existing law, the Corporate Securities L...","For purposes of this section, a “finder” is a ..."
171,The California Constitution generally limits a...,For purposes of subdivision (b) of Section 2 o...
1005,"Existing law, until January 1, 2021, establish...",The California Workforce Development Board sha...
978,"Existing law, known as the Ortiz-Pacheco-Pooch...",Cal Grant Program awards shall be known as “Ca...


In [ ]:
def RougeScore(ModelScore, ModelSummary):

    standard_summary = data["summary"]
    ModelScore_ = rouge.get_scores(ModelSummary, standard_summary, avg=True)
    ModelDF = pd.DataFrame(ModelScore_).set_index(
        [["recall", "precision", "f-measure"]]
    )
    ModelDF.to_csv(score_path + ModelScore + '.csv', index=True, header=True)
    return ModelDF

In [ ]:
LuhnRouge = RougeScore("LuhnRouge", data["LuhnSummary"])

In [ ]:
LuhnRouge

,rouge-1,rouge-2,rouge-l
recall,0.402877,0.214916,0.357938
precision,0.379553,0.208278,0.338494
f-measure,0.374886,0.199367,0.333564
